In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import DataFrame as df
import csv

#크롤링 탐지 우회 헤더 
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
pages = []
urls = []
for i in range(1, 77): #페이지 1 ~ n 추가 #다음에서 페이지 수 확인 필수!!
    pages.append\
    ("https://search.daum.net/search?nil_suggest=btn&w=blog&DA=PGD&q=%EC%9D%B4%EB%94%94%EC%95%BC&page="\
                 + str(i))

for page in pages: #페이지에서 url 가져오기
    html = requests.get(page, headers = headers).text
    soup = BeautifulSoup(html, 'html.parser')
    
    for link in soup.findAll('a', class_="f_link_b"):
        if 'href' in link.attrs:
            urls.append(link.attrs['href'])
#url testing
#print(urls)

In [10]:
#url에서 텍스트 크롤링

strings = [] 
for url in urls:

    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    nameList = soup.findAll('p')

    buff = []
    for name in nameList:
        buff.append(name.get_text())
    strings.append(''.join(buff))

    del buff #free buffer
    

AttributeError: 'bytes' object has no attribute 'timeout'

In [ ]:
#url에서 저자와 날짜 크롤링

authors = []
dates =  []

for url in urls:
#     print(url)
    htmlL = urlopen(url)
    soupL =BeautifulSoup(htmlL, 'html.parser')
    
    #get author
    if soupL.findAll('span', class_="author"):
        author = soupL.find('span', class_="author").get_text()        
    elif soupL.findAll('span', class_="info_post"):
        author = soupL.find('span', class_="info_post").get_text()        
    elif soupL.findAll('h1', class_="jb-site-title"):
        author = soupL.find('h1', class_="jb-site-title").get_text()        
    elif soupL.findAll('button', class_="btn_name"):
        author = soupL.find('button', class_="btn_name").get_text()        
    elif soupL.findAll('span', class_="txt_style"):
        author = soupL.find('span', class_="txt_style").get_text()        
    elif soupL.findAll('span', class_="txt_bar"):
        author = soupL.find('span', class_="txt_bar").get_text()        
    elif soupL.findAll('span', class_="p-author h-card"):
        author = soupL.find('span', class_="p-author h-card").get_text()        
    elif soupL.findAll('div', class_="blank"):
        author = soupL.find('div', class_="blank").get_text()        
    elif soupL.findAll('span', itemprop="name"):
        author = soupL.find('span', itemprop="name").get_text() 
    elif soupL.findAll('span', class_="title_text"):
        author = soupL.find('span', class_="title_text").get_text()                            
    elif soupL.findAll('div', class_="name"):
        author = soupL.find('div', class_="name").get_text()        
    elif soupL.findAll('a', class_="fade-link"):
        author = soupL.find('a', class_="fade-link").get_text()
    elif soupL.findAll('div', class_="nick"):
        author = soupL.find('div', class_="nick").get_text()
        
    else:
        author = "## AUTHOR NULL ##"
        
    #get date
    if soupL.findAll(class_="date"):
        date = soupL.find(class_="date").get_text()
    elif soupL.findAll(class_="text_bar"):
        date = soupL.find(class_="text_bar").get_text()   
    elif soupL.findAll('li', class_="digit"):
        date = soupL.find('li', class_="digit").get_text() 
    elif soupL.findAll('span', class_="jb-article-information-date"):
        date = soupL.find('span', class_="jb-article-information-date").get_text() 
    else: 
        date = "## DATE NULL ##"

        
    author = author.replace('\n', '').replace('\t','')
    date = date.replace('\n', '').replace('\t','')
    authors.append(author)
    dates.append(date)

In [ ]:
data = pd.DataFrame(columns = ['author', 'date', 'text']) #데이터프레임화

data['author'] = authors
data['date'] = dates
data['text'] = strings


# UTF-8 인코딩으로 저장 후 .txt 파일로 load 후 ANSI 인코딩 .CSV 파일로 저장!
data.to_csv('kw.ediya.csv') #데이터프레임 저장

#data